In [258]:
import os
import logging
#os.chdir('---')
#####################################################################################
### Section A: Please change this section to import necessary files and packages ###
#####################################################################################
import pandas as pd
import numpy as np
import time
from CPC15_BEASTpred import CPC15_BEAST_individual_pred

import time
import multiprocessing 


import numpy as np
from CPC18_getDist import CPC18_getDist
from CPC15_BEASTsimulation import CPC15_BEASTsimulation
import os
import logging
import time

import numpy as np
from CPC15_isStochasticDom import CPC15_isStochasticDom
from distSample import distSample

import numpy as np
from CPC15_isStochasticDom import CPC15_isStochasticDom
from distSample import distSample

In [15]:
Data = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/1_games_10K/Synth10K2.csv')

In [16]:
Data['GameId'] = range(1, len(Data) + 1)


In [68]:
Data['B1'] = 0
    
Data['sigma'] = 0
Data['kapa'] = 0
Data['beta']= 0 
Data['gama'] = 0 
Data['psi'] = 0
Data['theta'] = 0
Data['wamb'] = 0

Data['Beast1'] = 0


In [17]:
# Total number of Problem_individual to predict
nProblems = Data['GameId'].unique()
### End of Section A ###

In [19]:
i = 1
nProblems_i = nProblems[i*500 : 500*(i+1)]
len(nProblems_i)

500

In [71]:
game = nProblems_i[200]
# read problem's parameters
Ha = Data.loc[Data.GameId == game, 'Ha'].values[0]
pHa = Data.loc[Data.GameId == game, 'pHa'].values[0]
La = Data.loc[Data.GameId == game, 'La'].values[0]
LotShapeA = Data.loc[Data.GameId == game, 'LotShapeA'].values[0]
LotNumA = Data.loc[Data.GameId == game, 'LotNumA'].values[0]
Hb = Data.loc[Data.GameId == game, 'Hb'].values[0]
pHb = Data.loc[Data.GameId == game]['pHb'].values[0]
Lb = Data.loc[Data.GameId == game]['Lb'].values[0]
LotShapeB = Data.loc[Data.GameId == game]['LotShapeB'].values[0]
LotNumB = Data.loc[Data.GameId == game]['LotNumB'].values[0]
Amb = Data.loc[Data.GameId == game]['Amb'].values[0]
Corr = Data.loc[Data.GameId == game]['Corr'].values[0]

In [33]:
# Pred

# get both options' distributions
DistA = CPC18_getDist(Ha, pHa, La, LotShapeA, LotNumA)
DistB = CPC18_getDist(Hb, pHb, Lb, LotShapeB, LotNumB)


In [34]:
p = 0
for sigma in [float(round(0.5*7/3,2)), float(round(1.5*7/3,2)), float(round(2.5*7/3,2))]:
        for beta in [float(round(0.5*2.6/3,2)), float(round(1.5*2.6/3, 2)), float(round(2.5*2.6/3, 2))] : 
            for kapa in [1,2,3] : 
                for gama in [float(round(0.5*3,2)), float(round(2*0.5/3,2))] :
                    p = p+1
print(p)

54


In [35]:
sigma=[float(round(0.5*7/3,2)), float(round(1.5*7/3,2)), float(round(2.5*7/3,2))][0]
beta= [float(round(0.5*2.6/3,2)), float(round(1.5*2.6/3, 2)), float(round(2.5*2.6/3, 2))][0] 
kapa =[1,2,3][0]
gama= [float(round(0.5*3,2)), float(round(2*0.5/3,2))] [0]

In [36]:
# Simulation 
SIGMA = 7
KAPA = 3
BETA = 2.6
GAMA = 0.5
PSI = 0.07
THETA = 1

nTrials = 5
firstFeedback = 6
nBlocks = 1

# draw personal traits
#     sigma = SIGMA * np.random.uniform(size=1)
#     kapa = np.random.choice(range(1, KAPA+1), 1)
#     beta = BETA * np.random.uniform(size=1)
#     gama = GAMA * np.random.uniform(size=1)
psi = PSI * np.random.uniform(size=1)
theta = THETA * np.random.uniform(size=1)
wamb = 0

ObsPay = np.zeros(shape=(nTrials - firstFeedback + 1, 2))  # observed outcomes in A (col1) and B (col2)

# Useful variables
nA = DistA.shape[0]  # num outcomes in A
nB = DistB.shape[0]  # num outcomes in B

ambiguous = False

nfeed = 0  # "t"; number of outcomes with feedback so far

MinA = DistA[0, 0]
MinB = DistB[0, 0]
MaxOutcome = np.maximum(DistA[nA - 1, 0], DistB[nB - 1, 0])
SignMax = np.sign(MaxOutcome)

if MinA == MinB:
    RatioMin = 1
elif np.sign(MinA) == np.sign(MinB):
    RatioMin = min(abs(MinA), abs(MinB)) / max(abs(MinA), abs(MinB))
else:
    RatioMin = 0

Range = MaxOutcome - min(MinA, MinB)
BEVa = np.matrix.dot(DistA[:, 0], DistA[:, 1])

pEstB = DistB[:, 1]
BEVb = np.matrix.dot(DistB[:, 0], pEstB)

return_all =[]

for sigma in [float(round(0.5*7/3,2)), float(round(1.5*7/3,2)), float(round(2.5*7/3,2))]:
    for beta in [float(round(0.5*2.6/3,2)), float(round(1.5*2.6/3, 2)), float(round(2.5*2.6/3, 2))] : 
        for kapa in [1,2,3] : 
            for gama in [float(round(0.5*3,2)), float(round(2*0.5/3,2))] : 
                Decision = np.empty(shape=(nTrials, 1))
#                     simPred = np.empty(shape=(nBlocks, 1))

                pBias = np.array([beta / (beta + 1 + pow(nfeed, theta))])

                trivial = CPC15_isStochasticDom(DistA, DistB)

                # simulation of decisions
                for trial in range(nTrials):
                    STa = 0
                    STb = 0
                    # mental simulations
                    for s in range(1, kapa+1):
                        rndNum = np.random.uniform(size=2)
                        if rndNum[0] > pBias[nfeed]:  # Unbiased technique
                            if nfeed == 0:
                                outcomeA = distSample(DistA[:, 0], DistA[:, 1], rndNum[1])
                                outcomeB = distSample(DistB[:, 0], pEstB, rndNum[1])
                            else:
                                uniprobs = np.repeat([1 / nfeed], nfeed)
                                outcomeA = distSample(ObsPay[0:nfeed, 0], uniprobs, rndNum[1])
                                outcomeB = distSample(ObsPay[0:nfeed, 1], uniprobs, rndNum[1])

                        elif rndNum[0] > (2 / 3) * pBias[nfeed]:  # uniform
                            outcomeA = distSample(DistA[:, 0], np.repeat([1 / nA], nA), rndNum[1])
                            outcomeB = distSample(DistB[:, 0], np.repeat([1 / nB], nB), rndNum[1])

                        elif rndNum[0] > (1 / 3) * pBias[nfeed]:  # contingent pessimism
                            if SignMax > 0 and RatioMin < gama:
                                outcomeA = MinA
                                outcomeB = MinB
                            else:
                                outcomeA = distSample(DistA[:, 0], np.repeat([1 / nA], nA), rndNum[1])
                                outcomeB = distSample(DistB[:, 0], np.repeat([1 / nB], nB), rndNum[1])

                        else:  # Sign
                            if nfeed == 0:
                                outcomeA = Range * distSample(np.sign(DistA[:, 0]), DistA[:, 1], rndNum[1])
                                outcomeB = Range * distSample(np.sign(DistB[:, 0]), pEstB, rndNum[1])
                            else:
                                uniprobs = np.repeat([1 / nfeed], nfeed)
                                outcomeA = Range * distSample(np.sign(ObsPay[0:nfeed, 0]), uniprobs, rndNum[1])
                                outcomeB = Range * distSample(np.sign(ObsPay[0:nfeed, 1]), uniprobs, rndNum[1])

                        STa = STa + outcomeA
                        STb = STb + outcomeB

                    STa = STa / kapa
                    STb = STb / kapa

                    # error term
                    if trivial['dom'][0]:
                        error = 0
                    else:
                        error = sigma * np.random.normal(size=1)  # positive values contribute to attraction to A

                    # decision
                    Decision[trial] = (BEVa - BEVb) + (STa - STb) + error < 0
                    if (BEVa - BEVb) + (STa - STb) + error == 0:
                        Decision[trial] = np.random.choice(range(1, 3), size=1, replace=False) - 1

                # compute B-rates for this simulation

                simPred = np.mean(Decision[0:5])

                return_all.append([simPred, sigma, kapa, beta, gama, psi, theta, wamb] )



In [37]:
len(return_all)

54

In [38]:
simPred = return_all

In [39]:
simPred[0]

[0.6, 1.17, 1, 0.43, 1.5, array([0.00741862]), array([0.09125924]), 0]

In [16]:
avg_Prediction = np.repeat([0], 1) # change 5 to 1 => only 1 block 
avg_Prediction.shape = (1, 1) # change 5 to 1 => only 1 block 



In [40]:
avg_Prediction = np.repeat([0], 1) # change 5 to 1 => only 1 block 
avg_Prediction.shape = (1, 1) # change 5 to 1 => only 1 block 

all_Prediction = [[]]*200*54  # change 5 to 1 => only 1 block 

# get both options' distributions
DistA = CPC18_getDist(Ha, pHa, La, LotShapeA, LotNumA)
DistB = CPC18_getDist(Hb, pHb, Lb, LotShapeB, LotNumB)

# run model simulation nSims times
nSims = 200
for sim in range(0, nSims):
    if sim % 20 == 0 : 
        logging.info('{}: Finish sim number : {}'.format((time.asctime(time.localtime(time.time()))), sim))
    simPred = CPC15_BEASTsimulation(DistA, DistB, Amb, Corr)
    for t in range(54) :
        avg_Prediction = np.add(avg_Prediction, (1 / (nSims*54)) * simPred[t][0])
        simPred[t][0] = round((simPred[t][0])*5)/5
        all_Prediction[sim*54 +t] = simPred[t]


In [41]:
avg_Prediction_Prediction

array([[0.44457407]])

In [42]:
len(all_Prediction)

10800

In [43]:
len(all_Prediction[300])

8

In [44]:
preddf = pd.DataFrame.from_records(all_Prediction)

In [45]:
preddf.shape

(10800, 8)

In [46]:
preddf.head()

,0,1,2,3,4,5,6,7
0,0.6,1.17,1,0.43,1.50,[0.008772935466397108],[0.22643040015013538],0
1,0.4,1.17,1,0.43,0.33,[0.008772935466397108],[0.22643040015013538],0
2,0.4,1.17,2,0.43,1.50,[0.008772935466397108],[0.22643040015013538],0
3,0.8,1.17,2,0.43,0.33,[0.008772935466397108],[0.22643040015013538],0
4,0.4,1.17,3,0.43,1.50,[0.008772935466397108],[0.22643040015013538],0


In [47]:
preddf.columns = ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']

In [48]:
np.mean(preddf.B1)

0.4445740740740635

In [49]:
preddf.B1.value_counts()

0.4    3574
0.6    2827
0.2    2321
0.8    1198
0.0     627
1.0     253
Name: B1, dtype: int64

In [50]:
preddf.sigma.value_counts()

1.17    3600
5.83    3600
3.50    3600
Name: sigma, dtype: int64

In [51]:
Data.loc[(Data['GameId'] == game), ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']].shape

(1, 8)

In [52]:
rep_prob.shape

(10800, 22)

In [54]:
Data2 = Data
print(Data.columns)

Index(['Ha', 'pHa', 'La', 'LotNumA', 'LotShapeA', 'Hb', 'pHb', 'Lb', 'LotNumB',
       'LotShapeB', 'Corr', 'Amb', 'GameId', 'B1', 'sigma', 'kapa', 'beta',
       'gama', 'psi', 'theta', 'wamb', 'Beast1'],
      dtype='object')


In [55]:
Data.head()

,Ha,pHa,La,LotNumA,LotShapeA,Hb,pHb,Lb,LotNumB,LotShapeB,...,GameId,B1,sigma,kapa,beta,gama,psi,theta,wamb,Beast1
0,53,0.25,-20,1,-,86,0.25,-23,1,-,...,1,0,0,0,0,0,0,0,0,0
1,42,0.01,9,4,R-skew,9,1.00,9,1,-,...,2,0,0,0,0,0,0,0,0,0
2,-4,1.00,-4,1,-,5,0.99,-20,4,Symm,...,3,0,0,0,0,0,0,0,0,0
3,13,1.00,13,1,-,54,0.40,-9,1,-,...,4,0,0,0,0,0,0,0,0,0
4,-6,1.00,-6,1,-,-6,1.00,-6,8,L-skew,...,5,0,0,0,0,0,0,0,0,0


In [56]:
for game in nProblems_i[:2]:
    # Replicate the Problem 200 times, for 200 participants in each problem 
    dfdf = Data.loc[(Data.GameId == game)] 
    rep_prob = pd.DataFrame(np.repeat(dfdf.values, int(200*54), axis = 0), columns= Data.columns)
    Data = pd.concat([Data, rep_prob])


In [59]:
dfdf = Data2.loc[(Data2.GameId == game)] 
rep_prob = pd.DataFrame(np.repeat(dfdf.values, int(200*54), axis = 0), columns= Data2.columns)
Data2 = pd.concat([Data2, rep_prob])


In [75]:
Data= Data2

In [76]:
game = nProblems_i[200]
dfdf = Data.loc[(Data.GameId == game)] 
rep_prob = pd.DataFrame(np.repeat(dfdf.values, int(200*54)-1, axis = 0), columns= Data.columns)
Data = pd.concat([Data, rep_prob])
Data.loc[(Data['GameId'] == game), ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']] = pd.DataFrame.from_records(all_Prediction)


In [81]:
pd.DataFrame.from_records(all_Prediction).shape

(10800, 8)

In [82]:
Data.loc[(Data['GameId'] == game), ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']].shape

(10800, 8)

In [88]:
pred_def = pd.DataFrame.from_records(all_Prediction)

In [89]:
pred_def.shape

(10800, 8)

In [91]:
pred_def.columns = ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']

In [83]:
Data.loc[(Data['GameId'] == game), ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']] = pd.DataFrame.from_records(all_Prediction)

In [93]:
Data.loc[(Data['GameId'] == game), 'B1'] = pred_def['B1']

In [94]:
Data.loc[(Data['GameId'] == game), 'B1']

200      0.8
0        0.6
1        0.4
2        0.4
3        0.8
4        0.4
5          0
6        0.2
7        0.8
8        0.4
9        0.8
10       0.4
11       0.6
12       0.2
13       0.2
14       0.4
15       0.8
16       0.8
17       0.6
18       0.2
19       0.6
20       0.4
21       0.8
22       0.2
23       0.2
24       0.2
25         0
26       0.4
27       0.8
28       0.8
        ... 
10769      1
10770    0.4
10771    0.2
10772    0.4
10773    0.2
10774    0.4
10775    0.4
10776    0.4
10777    0.6
10778    0.4
10779    0.4
10780    0.4
10781    0.4
10782    0.2
10783    0.6
10784    0.4
10785    0.8
10786    0.6
10787    0.4
10788    0.6
10789    0.2
10790    0.4
10791    0.4
10792    0.6
10793      0
10794    0.2
10795    0.4
10796    0.4
10797    0.2
10798    0.2
Name: B1, Length: 10800, dtype: object

In [3]:
# load problems to predict 
Data = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/1_games_10K/Synth10K2.csv')
Data['GameId'] = range(1, len(Data) + 1)

# Add the "ground thruth = Beast_individual" for every participant in evry block : 
Data['B1'] = 0

Data['sigma'] = 0
Data['kapa'] = 0
Data['beta']= 0 
Data['gama'] = 0 
Data['psi'] = 0
Data['theta'] = 0
Data['wamb'] = 0

Data['Beast1'] = 0

# Total number of Problem_individual to predict
nProblems = Data['GameId'].unique()

In [8]:
i = 0
nProblems_i = nProblems[i*500 : 500*(i+1)]
len(nProblems_i)

500

In [4]:
Data.columns

Index(['Ha', 'pHa', 'La', 'LotNumA', 'LotShapeA', 'Hb', 'pHb', 'Lb', 'LotNumB',
       'LotShapeB', 'Corr', 'Amb', 'GameId', 'B1', 'sigma', 'kapa', 'beta',
       'gama', 'psi', 'theta', 'wamb', 'Beast1'],
      dtype='object')

In [5]:
Data.shape

(10000, 22)

In [9]:
game = nProblems_i[200]
# read problem's parameters
Ha = Data.loc[Data.GameId == game, 'Ha'].values[0]
pHa = Data.loc[Data.GameId == game, 'pHa'].values[0]
La = Data.loc[Data.GameId == game, 'La'].values[0]
LotShapeA = Data.loc[Data.GameId == game, 'LotShapeA'].values[0]
LotNumA = Data.loc[Data.GameId == game, 'LotNumA'].values[0]
Hb = Data.loc[Data.GameId == game, 'Hb'].values[0]
pHb = Data.loc[Data.GameId == game]['pHb'].values[0]
Lb = Data.loc[Data.GameId == game]['Lb'].values[0]
LotShapeB = Data.loc[Data.GameId == game]['LotShapeB'].values[0]
LotNumB = Data.loc[Data.GameId == game]['LotNumB'].values[0]
Amb = Data.loc[Data.GameId == game]['Amb'].values[0]
Corr = Data.loc[Data.GameId == game]['Corr'].values[0]

In [10]:
# Use Beast to give the ground thruth : modify beast : only 50 simulations for every prediction not 50000
(all_prediction, Prediction) = CPC15_BEAST_individual_pred(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr)

#Add the prediction to the Data
Data.loc[Data.GameId == game, 'Beast1'] = Prediction[0]

# Replicate the Problem 200 times, for 200 participants in each problem 
dfdf = Data.loc[(Data.GameId == game)] 
rep_prob = pd.DataFrame(np.repeat(dfdf.values, int(200*54)-1, axis = 0), columns= Data.columns)
Data = pd.concat([Data, rep_prob])
all_pred_df= pd.DataFrame.from_records(all_prediction)
all_pred_df.columns = ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']
for col in ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb'] : 
    Data.loc[(Data['GameId'] == game), col] = all_pred_df[col]

In [12]:
Data.loc[(Data['GameId'] == game)].shape

(10800, 22)

In [2]:
import pandas as pd
synth1 = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/3_Generate_Answers /synth_allV2/synth_1.csv')

In [3]:
synth1.shape

(5409500, 22)

In [4]:
synth1.head()

,Ha,pHa,La,LotNumA,LotShapeA,Hb,pHb,Lb,LotNumB,LotShapeB,...,GameId,B1,sigma,kapa,beta,gama,psi,theta,wamb,Beast1
0,53,0.25,-20,1,-,86,0.25,-23,1,-,...,1,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
1,42,0.01,9,4,R-skew,9,1.00,9,1,-,...,2,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
2,-4,1.00,-4,1,-,5,0.99,-20,4,Symm,...,3,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
3,13,1.00,13,1,-,54,0.40,-9,1,-,...,4,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0
4,-6,1.00,-6,1,-,-6,1.00,-6,8,L-skew,...,5,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0.0


In [14]:
pd.DataFrame(synth1.GameId.value_counts())['GameId'].value_counts()

1        9500
10800     500
Name: GameId, dtype: int64

In [31]:
pd.DataFrame(synth1.gama.value_counts())
# pd.DataFrame(synth1.loc[synth1.GameId.isin(nProblems_i)].beta.value_counts())

,gama
1.50,2700250
0.33,2699750
0.00,9500


In [36]:
200*500*54

5400000

In [37]:
for i in range(20) : 
    synth_i = pd.read_csv(str('/home/meghanmergui/Synth20/Create_Synthetic_Games/3_Generate_Answers /synth_allV2/synth_') +str(i) +str('.csv'))
    nProblems_i = nProblems[i*500 : 500*(i+1)]
    synth_i = synth_i.loc[synth_i.GameId.isin(nProblems_i)]
    print(pd.DataFrame(synth_i.loc[synth_i.GameId.isin(nProblems_i)].GameId.value_counts())['GameId'].value_counts())
    print(synth_i.shape)

10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, d

In [39]:
for i in range(20) : 
    synth_i = pd.read_csv(str('/home/meghanmergui/Synth20/Create_Synthetic_Games/3_Generate_Answers /synth_allV2/synth_') +str(i) +str('.csv'))
    nProblems_i = nProblems[i*500 : 500*(i+1)]
    synth_i = synth_i.loc[synth_i.GameId.isin(nProblems_i)]
    print(pd.DataFrame(synth_i.loc[synth_i.GameId.isin(nProblems_i)].GameId.value_counts())['GameId'].value_counts())
    print(synth_i.shape)
    synth_i.to_csv(str('/home/meghanmergui/Synth20/Create_Synthetic_Games/3_Generate_Answers /synth_all/synth_') +str(i) +str('.csv'), index = False)

10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, dtype: int64
(5400000, 22)
10800    500
Name: GameId, d

# Check synth 500 

In [3]:
synth10 = pd.read_csv("/home/meghanmergui/Synth20/Create_Synthetic_Games/3_Generate_Answers /synth_all/synth_10.csv")

In [4]:
synth10.GameId.nunique()

500

In [44]:
synth10[['sigma', 'beta', 'kapa', 'gama']].drop_duplicates().shape

(54, 4)

In [45]:
synth10[['sigma', 'beta', 'kapa', 'gama', 'GameId']].drop_duplicates().shape

(27000, 5)

In [46]:
54*500

27000

In [5]:
aa = pd.DataFrame(synth10.groupby(['sigma', 'beta', 'kapa', 'gama'])['GameId'].nunique())

aa.GameId.value_counts()

500    54
Name: GameId, dtype: int64

In [7]:
a = pd.DataFrame(synth10.groupby(['sigma', 'beta', 'kapa', 'gama', 'GameId'])['GameId'].count())
print(a.head())
a.GameId.value_counts()

                             GameId
sigma beta kapa gama GameId        
1.17  0.43 1    0.33 5001       200
                     5002       200
                     5003       200
                     5004       200
                     5005       200


200    26018
201      491
199      491
Name: GameId, dtype: int64

In [11]:
a.loc[a.GameId==199]

GameId
sigma beta kapa gama GameId        
5.83  2.17 3    0.33 5001       199
                     5002       199
                     5003       199
                     5004       199
                     5005       199
                     5006       199
                     5007       199
                     5008       199
                     5009       199
                     5010       199
                     5011       199
                     5012       199
                     5013       199
                     5014       199
                     5015       199
                     5016       199
                     5017       199
                     5018       199
                     5019       199
                     5020       199
                     5021       199
                     5023       199
                     5024       199
                     5025       199
                     5026       199
                     5027       199
                     5028       199
                     5029       199
                     5030       199
                     5031       199
...                             ...
                     5471       199
                     5472       199
                     5473       199
                     5474       199
                     5475       199
                     5476       199
                     5477       199
                     5478       199
                     5479       199
                     5480       199
                     5481       199
                     5482       199
                     5483       199
                     5484       199
                     5485       199
                     5486       199
                     5487       199
                     5488       199
                     5489       199
                     5490       199
                     5491       199
                     5492       199
                     5493       199
                     5494       199
                     5495       199
                     5496       199
                     5497       199
                     5498       199
                     5499       199
                     5500       199

[491 rows x 1 columns]

In [62]:
26018/(26018+491+491)

0.9636296296296296

In [49]:
aa.GameId.value_counts()

500    54
Name: GameId, dtype: int64

In [72]:
f = synth10[['GameId', 'sigma', 'beta', 'kapa', 'gama']]

In [73]:
f['type'] = f.apply(lambda x : str(str(x.sigma) + str(x.beta) + str(x.kapa) + str(x.gama)), axis = 1)

/home/meghanmergui@st.technion.ac.il/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [74]:
f.groupby(['GameId', 'type'])['type'].count()

GameId  type           
5001    1.170.431.00.33    200
        1.170.431.01.5     200
        1.170.432.00.33    200
        1.170.432.01.5     200
        1.170.433.00.33    200
        1.170.433.01.5     200
        1.171.31.00.33     200
        1.171.31.01.5      200
        1.171.32.00.33     200
        1.171.32.01.5      200
        1.171.33.00.33     200
        1.171.33.01.5      200
        1.172.171.00.33    200
        1.172.171.01.5     200
        1.172.172.00.33    200
        1.172.172.01.5     200
        1.172.173.00.33    200
        1.172.173.01.5     200
        3.50.431.00.33     200
        3.50.431.01.5      200
        3.50.432.00.33     200
        3.50.432.01.5      200
        3.50.433.00.33     200
        3.50.433.01.5      200
        3.51.31.00.33      200
        3.51.31.01.5       200
        3.51.32.00.33      200
        3.51.32.01.5       200
        3.51.33.00.33      200
        3.51.33.01.5       200
                          ... 
5500    3.51.31

In [64]:
b = synth10[['GameId', 'sigma', 'beta', 'kapa', 'gama']].drop_duplicates()
b.groupby(['sigma', 'beta', 'kapa', 'gama']).GameId.count()

sigma  beta  kapa  gama
1.17   0.43  1     0.33    500
                   1.50    500
             2     0.33    500
                   1.50    500
             3     0.33    500
                   1.50    500
       1.30  1     0.33    500
                   1.50    500
             2     0.33    500
                   1.50    500
             3     0.33    500
                   1.50    500
       2.17  1     0.33    500
                   1.50    500
             2     0.33    500
                   1.50    500
             3     0.33    500
                   1.50    500
3.50   0.43  1     0.33    500
                   1.50    500
             2     0.33    500
                   1.50    500
             3     0.33    500
                   1.50    500
       1.30  1     0.33    500
                   1.50    500
             2     0.33    500
                   1.50    500
             3     0.33    500
                   1.50    500
       2.17  1     0.33    500
               

In [66]:
b['type'] = b.apply(lambda x : str(str(x.sigma) + str(x.beta) + str(x.kapa) + str(x.gama)), axis = 1)

In [69]:
b.groupby('GameId')['type'].count().unique()

array([54])

In [51]:
bb = pd.DataFrame(synth10.groupby(['sigma', 'beta', 'kapa', 'gama'])['GameId'].count())
bb

GameId
sigma beta kapa gama        
1.17  0.43 1    0.33  100009
                1.50  100009
           2    0.33  100009
                1.50  100009
           3    0.33  100009
                1.50  100009
      1.30 1    0.33  100009
                1.50  100009
           2    0.33  100009
                1.50  100009
           3    0.33  100009
                1.50  100009
      2.17 1    0.33  100009
                1.50  100009
           2    0.33  100009
                1.50  100009
           3    0.33  100009
                1.50  100009
3.50  0.43 1    0.33  100009
                1.50  100009
           2    0.33  100009
                1.50  100009
           3    0.33  100009
                1.50  100009
      1.30 1    0.33  100009
                1.50  100009
           2    0.33  100009
                1.50  100009
           3    0.33  100009
                1.50  100009
      2.17 1    0.33  100009
                1.50  100009
           2    0.33  100010
                1.50  100010
           3    0.33  100010
                1.50  100010
5.83  0.43 1    0.33  100010
                1.50  100010
           2    0.33  100010
                1.50  100010
           3    0.33  100010
                1.50  100010
      1.30 1    0.33  100010
                1.50  100010
           2    0.33  100010
                1.50  100010
           3    0.33  100009
                1.50  100009
      2.17 1    0.33  100009
                1.50  100009
           2    0.33  100009
                1.50  100009
           3    0.33   99509
                1.50  100009

In [53]:
cc = pd.DataFrame(synth10.groupby(['sigma', 'beta', 'kapa'])['GameId'].count())
cc

GameId
sigma beta kapa        
1.17  0.43 1     200018
           2     200018
           3     200018
      1.30 1     200018
           2     200018
           3     200018
      2.17 1     200018
           2     200018
           3     200018
3.50  0.43 1     200018
           2     200018
           3     200018
      1.30 1     200018
           2     200018
           3     200018
      2.17 1     200018
           2     200020
           3     200020
5.83  0.43 1     200020
           2     200020
           3     200020
      1.30 1     200020
           2     200020
           3     200018
      2.17 1     200018
           2     200018
           3     199518

In [55]:
dd = pd.DataFrame(synth10.groupby(['kapa'])['GameId'].count())
dd

,GameId
kapa,
1,1800166
2,1800168
3,1799666


# Check synthetic kapa

In [296]:
for i in range(20) : 
    synth_kapa = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/3_Generate_Answers /synth_all_kapa/synth_' + str(i) +'.csv')
    print(synth_kapa.kapa.unique())
    a = pd.DataFrame(synth_kapa.groupby(['kapa', 'GameId'])['GameId'].count())
    print(a.GameId.value_counts())

[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64
[1 2 3]
200    1500
Name: GameId, dtype: int64


In [287]:
synth_kapa.GameId.nunique()

500

In [288]:
synth_kapa.shape

(300000, 22)

In [289]:
synth_kapa.kapa.unique()

array([1, 2, 3])

In [7]:
3*200*10000


6000000

In [290]:
synth_kapa[['sigma', 'beta', 'kapa', 'gama']].drop_duplicates().shape

(3, 4)

In [291]:
synth_kapa.loc[~(synth_kapa['kapa'].isin([1,2,3]))].shape

(0, 22)

In [292]:
synth_kapa[['GameId', 'kapa']].drop_duplicates().shape

(1500, 2)

In [9]:
3*500

1500

In [293]:
aa = pd.DataFrame(synth_kapa.groupby(['sigma', 'beta', 'gama', 'kapa'])['GameId'].count())

aa.GameId.value_counts()

100000    3
Name: GameId, dtype: int64

In [294]:
aa

GameId
sigma beta gama kapa        
3.5   1.3  0.25 1     100000
                2     100000
                3     100000

In [295]:
a = pd.DataFrame(synth_kapa.groupby(['kapa', 'GameId'])['GameId'].count())
print(a.head())
a.GameId.value_counts()

             GameId
kapa GameId        
1    5001       200
     5002       200
     5003       200
     5004       200
     5005       200


200    1500
Name: GameId, dtype: int64

In [43]:
b = a.loc[a.GameId.isin([199, 201])]
b.columns = ['counts']
b.reset_index( inplace = True)

In [79]:
b.loc[b.GameId == 7]

,kapa,GameId,counts
2,1,7,201
338,3,7,199


In [280]:
Data = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/1_games_10K/Synth10K2.csv')
Data['GameId'] = range(1, len(Data) + 1)
nProblems = Data['GameId'].unique()
i = 3
nProblems_i = nProblems[i*500 : 500*(i+1)]


In [281]:
Data.loc[Data.GameId.isin(nProblems_i)].shape

(500, 13)

In [282]:
game = nProblems_i[0] 
# read problem's parameters
Ha = Data.loc[Data.GameId == game, 'Ha'].values[0]
pHa = Data.loc[Data.GameId == game, 'pHa'].values[0]
La = Data.loc[Data.GameId == game, 'La'].values[0]
LotShapeA = Data.loc[Data.GameId == game, 'LotShapeA'].values[0]
LotNumA = Data.loc[Data.GameId == game, 'LotNumA'].values[0]
Hb = Data.loc[Data.GameId == game, 'Hb'].values[0]
pHb = Data.loc[Data.GameId == game]['pHb'].values[0]
Lb = Data.loc[Data.GameId == game]['Lb'].values[0]
LotShapeB = Data.loc[Data.GameId == game]['LotShapeB'].values[0]
LotNumB = Data.loc[Data.GameId == game]['LotNumB'].values[0]
Amb = Data.loc[Data.GameId == game]['Amb'].values[0]
Corr = Data.loc[Data.GameId == game]['Corr'].values[0]


In [283]:
(all_prediction, Prediction) = CPC15_BEAST_individual_pred(Ha, pHa, La, LotShapeA, LotNumA, Hb, pHb, Lb, LotShapeB, LotNumB, Amb, Corr)

In [284]:
Data.loc[Data.GameId == game]

,Ha,pHa,La,LotNumA,LotShapeA,Hb,pHb,Lb,LotNumB,LotShapeB,Corr,Amb,GameId
1500,61,0.4,-7,7,L-skew,44,0.01,21,2,Symm,0,0,1501


In [285]:
Data.loc[Data.GameId == game, 'Beast1'] = Prediction[0]

In [265]:
# Replicate the Problem 200 times, for 200 participants in each problem 
dfdf = Data.loc[(Data.GameId == game)].copy()
rep_prob = pd.DataFrame(np.repeat(dfdf.values, int(200*3)-1, axis = 0), columns= Data.columns)
Data = Data.append(rep_prob, ignore_index = True) 

In [266]:
Data.loc[Data.GameId == game].shape

(600, 14)

In [267]:
all_pred_df= pd.DataFrame.from_records(all_prediction)
all_pred_df.shape
all_pred_df.columns =  ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']


In [268]:
#         rep_prob = pd.DataFrame(np.repeat(dfdf.values, int(200*54)-1, axis = 0), columns= Data.columns)
for col in ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb'] :
    Data.loc[(Data['GameId'] == game), col] = pd.Series(list(all_pred_df[col])).values


In [269]:
Data.loc[(Data['GameId'] == game), ['B1', 'sigma', 'kapa', 'beta', 'gama', 'psi', 'theta', 'wamb']].shape

(600, 8)

In [270]:
Data.loc[Data.GameId == game].head()

,Ha,pHa,La,LotNumA,LotShapeA,Hb,pHb,Lb,LotNumB,LotShapeB,...,GameId,Beast1,B1,sigma,kapa,beta,gama,psi,theta,wamb
1500,61,0.4,-7,7,L-skew,44,0.01,21,2,Symm,...,1501,0.772,1.0,3.5,1.0,1.3,0.25,0.004856,0.969236,0.0
10000,61,0.4,-7,7,L-skew,44,0.01,21,2,Symm,...,1501,0.772,0.4,3.5,2.0,1.3,0.25,0.004856,0.969236,0.0
10001,61,0.4,-7,7,L-skew,44,0.01,21,2,Symm,...,1501,0.772,0.8,3.5,3.0,1.3,0.25,0.004856,0.969236,0.0
10002,61,0.4,-7,7,L-skew,44,0.01,21,2,Symm,...,1501,0.772,0.8,3.5,1.0,1.3,0.25,0.065254,0.597578,0.0
10003,61,0.4,-7,7,L-skew,44,0.01,21,2,Symm,...,1501,0.772,0.8,3.5,2.0,1.3,0.25,0.065254,0.597578,0.0


In [275]:
Data.loc[Data.GameId==game][['GameId', 'kapa']]['GameId'].count()

600

In [276]:
Data.loc[Data.GameId == game].shape

(600, 22)

In [279]:
Data.loc[Data.GameId == game].kapa.unique()

array([1., 2., 3.])

In [88]:
for i in range(1, 50 + 1) : 
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
